In [4]:
import pandas as pd
import requests

In [5]:
symbols_stock = pd.read_csv("list_stock.csv")

symbols_stock['Symbol'] = symbols_stock['Symbol'].str.replace('^', '')

display(symbols_stock.shape)

C:\Users\adamm\AppData\Local\Temp\ipykernel_7128\1761626554.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  symbols_stock['Symbol'] = symbols_stock['Symbol'].str.replace('^', '')


(5807, 1)